In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [3]:
symbols = ["MES", "6B"]
df = pth.SierraChartData.ffill(
    pth.SierraChartData.pull(
        symbols,
        timeframe="1min",
        start="2024-11-01",
        end="2024-12-01",
    )
)
df

c:\Users\spask\miniconda3\Lib\site-packages\vectorbtpro\data\base.py:1728: VBTWarning: Symbols have mismatching index. Setting missing data points to NaN.


In [4]:
close = df.data['MES'].close
close.vbt.plot()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a9627f23-ee74-49e0-897e-a1f26efdca8c',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'norm

### Generation

In [5]:
indicator = vbt.talib("BBANDS").run(
    df.get('close'),
    timeperiod=vbt.Default(14),
    nbdevup=vbt.Default(2),
    nbdevdn=vbt.Default(2),
)
indicator

In [6]:
symbol = 'MES'
fig = df.get('close', symbol).vbt.plot()
indicator.lowerband[symbol].vbt.plot(fig=fig)
indicator.upperband[symbol].vbt.plot(fig=fig)

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '78d5fc17-1042-4753-b5a4-e764de25b097',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])},
             {'name': 'MES',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b3a1

In [7]:
(df.get('close') < indicator.lowerband).sum()

symbol
MES    1479
6B     1462
dtype: int64

In [8]:
indicator = vbt.talib("BBANDS").run(
    close,
    timeperiod=vbt.Default(60),
    nbdevup=[2,3],
    nbdevdn=[2,3],
)
indicator

In [9]:
indicator.lowerband

bbands_nbdevup,2,3
bbands_nbdevdn,2,3
timestamp,,
2024-11-01 00:00:00+00:00,NaN,NaN
2024-11-01 00:01:00+00:00,NaN,NaN
2024-11-01 00:02:00+00:00,NaN,NaN
2024-11-01 00:03:00+00:00,NaN,NaN
2024-11-01 00:04:00+00:00,NaN,NaN
...,...,...
2024-12-01 23:55:00+00:00,6113.056345,6110.759517
2024-12-01 23:56:00+00:00,6113.034111,6110.780334


In [10]:
(close.vbt < indicator.lowerband).sum()

bbands_nbdevup  bbands_nbdevdn
2               2                 1995
3               3                  270
dtype: int64

In [11]:
# this is the same as above
indicator.lowerband_above(close).sum()

bbands_nbdevup  bbands_nbdevdn
2               2                 1995
3               3                  270
dtype: int64

In [12]:
bandwidth = (indicator.upperband - indicator.lowerband) / indicator.middleband
bandwidth

bbands_nbdevup,2,3
bbands_nbdevdn,2,3
timestamp,,
2024-11-01 00:00:00+00:00,NaN,NaN
2024-11-01 00:01:00+00:00,NaN,NaN
2024-11-01 00:02:00+00:00,NaN,NaN
2024-11-01 00:03:00+00:00,NaN,NaN
2024-11-01 00:04:00+00:00,NaN,NaN
...,...,...
2024-12-01 23:55:00+00:00,0.001502,0.002253
2024-12-01 23:56:00+00:00,0.001474,0.002210


In [13]:
# compare with list of params just adds more columns as expected (don't forget the .vbt  accessor!)
(bandwidth.vbt > vbt.Param([0.0075, 0.01], name='threshold')).sum()

threshold  bbands_nbdevup  bbands_nbdevdn
0.0075     2               2                  313
           3               3                 1296
0.0100     2               2                   62
           3               3                  534
dtype: int64

In [14]:
indicator = vbt.talib("BBANDS").run(
    df.get('close'),
    timeperiod=vbt.Default(60),
    nbdevup=[2,3],
    nbdevdn=[2,3],
)
indicator

In [15]:
mes = df.get('close', 'MES')
mes_indicator = indicator.lowerband[(2,2, 'MES')].rename('lowerband')
mes_bandwidth = bandwidth[(2,2)].rename('bandwidth')
signals = mes.vbt.crossed_below(
    mes_indicator,
    wait=1  # how many bars to wait for confirmation of the signal
)
signals.sum()

np.int64(347)

In [16]:
fig = mes.vbt.plot()
mes_indicator.vbt.plot(fig=fig)
signals.vbt.signals.plot_as_markers(
    mes,
    fig=fig,
    trace_kwargs=dict(marker=dict(color='#ff0000')) # optional
)

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b4077c0f-d405-41ed-bba1-4b9ed5d61790',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])},
             {'name': 'lowerband',
              'showlegend': True,
              'type': 'scatter',
              'uid':

In [17]:
# multiple conditions for signal generation
condition_lowerband = mes.vbt.crossed_below(mes_indicator)
condition_threshold = mes_bandwidth > 0.003
signals = condition_lowerband & condition_threshold
signals.sum()

np.int64(36)

In [18]:
fig = mes.vbt.plot()
mes_indicator.vbt.plot(fig=fig)
signals.vbt.signals.plot_as_markers(
    mes,
    fig=fig,
    trace_kwargs=dict(marker=dict(color='#ff0000')) # optional
)

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '30ad80cd-7b46-4ba2-a08b-d37f2eb1e99c',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])},
             {'name': 'lowerband',
              'showlegend': True,
              'type': 'scatter',
              'uid':

In [21]:
multi_cond = mes_bandwidth.vbt > vbt.Param([0.001, 0.002, 0.003, 0.005, 0.01], name='threshold')
multi_cond.sum()

threshold
0.001    18942
0.002     8612
0.003     4045
0.005     1296
0.010       62
dtype: int64

In [93]:
series = pd.Series([False, False, False, True, False, False])
series.vbt.rolling_any(2)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [108]:
hourly = df.data['MES'].iloc[df.wrapper.get_index_points(every='1h')]
hourly.close.vbt.plot()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '83e53d5a-5d50-42e4-8444-57a471650703',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 1, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 2, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5813.  , 5809.75, ..., 6123.25, 6123.25, 6123.25])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal'

### Time of Day (Periodic) Signals

In [213]:
mask = df.get_symbol_wrapper().fill(False)
mask

symbol,MES,6B
timestamp,,
2024-11-01 00:00:00+00:00,False,False
2024-11-01 00:01:00+00:00,False,False
2024-11-01 00:02:00+00:00,False,False
2024-11-01 00:03:00+00:00,False,False
2024-11-01 00:04:00+00:00,False,False
...,...,...
2024-12-01 23:55:00+00:00,False,False
2024-12-01 23:56:00+00:00,False,False
2024-12-01 23:57:00+00:00,False,False


In [216]:
entries = mask.vbt.set(True, every='monday', at_time='12:30')
entries[entries.any(axis=1)]

symbol,MES,6B
timestamp,,
2024-11-04 12:30:00+00:00,True,True
2024-11-11 12:30:00+00:00,True,True
2024-11-18 12:30:00+00:00,True,True
2024-11-25 12:30:00+00:00,True,True


In [217]:
exits = mask.vbt.set(True, every='friday', at_time='13:30')
exits[exits.any(axis=1)]

symbol,MES,6B
timestamp,,
2024-11-01 13:30:00+00:00,True,True
2024-11-08 13:30:00+00:00,True,True
2024-11-15 13:30:00+00:00,True,True
2024-11-22 13:30:00+00:00,True,True
2024-11-29 13:30:00+00:00,True,True


In [311]:
pf = vbt.Portfolio.from_signals(
    df.get('close'),
    long_entries=entries,
    long_exits=exits,
)
pf['MES'].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'daeec2b3-52ec-42d1-9e8d-dfb0eb9cb08c',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
           

In [314]:
mask = df.get_symbol_wrapper().fill(False)
long_entries = mask.vbt.set(True, every='1D', at_time='13:00')
long_exits = mask.vbt.set(True, every='1D', at_time='18:30')
pf = vbt.Portfolio.from_signals(
    df.get('close'),
    long_entries=long_entries,
    long_exits=long_exits,
    # short_entries=short_entries,
    # short_exits=short_exits
)
pf['MES'].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'e8325888-7e7f-44e0-9539-d5521ba0784e',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
           

### Iterative Signals

In [411]:
@njit(nogil=True)
def make_signals(close, lowerband, upperband):
    entries = np.full(close.shape[0], False)
    exits = np.full(close.shape[0], False)

    for i in range(close.shape[0]):
        if close[i] < lowerband[i]:
            entries[i] = True
        elif close[i] > upperband[i]:
            exits[i] = True
            
    return entries, exits

In [412]:
lowerband = indicator.lowerband[(3, 3, 'MES')]
upperband = indicator.upperband[(3, 3, 'MES')]

In [415]:
%%timeit
entries, exits = make_signals(close.values, lowerband.values, upperband.values)
entries = pd.Series(entries, index=close.index)
exits = pd.Series(exits, index=close.index)
exits.sum()

80.2 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [402]:
entries = entries.vbt.set_between(False, every='D', start_time='20:00', end_time='13:00')
exits = exits.vbt.set_between(False, every='D', start_time='20:00', end_time='13:00')
exits = exits.vbt.set(True, every='D', at_time='20:00')
entries.sum(), exits.sum()

(np.int64(85), np.int64(104))

In [404]:
pf = vbt.Portfolio.from_signals(
    close,
    long_entries=entries,
    long_exits=exits,
)
pf.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '3f2bca27-4786-4279-9e5d-33850cce4442',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
           

In [405]:
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '309cf0c1-2610-4948-a250-4b1fed911107',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116. 

### Generators

In [417]:
# TODO: continue here with generators when you're coming back to vectorbt.